In [ ]:
import requests

contests = requests.get("https://codeforces.com/api/contest.list").json()

In [ ]:
contests

In [ ]:
import requests

subs = requests.get("https://codeforces.com/api/contest.status?contestId=1500&from=1&count=10000").json()

In [ ]:
subs["result"][2]

In [ ]:
import requests

probs = requests.get("https://codeforces.com/api/problemset.problems")
print(probs)

In [ ]:
from lxml import html

tree = html.fromstring(req.text)

In [ ]:
tree.xpath('//*[@id="pageContent"]/div[3]/pre/text()')[0]

In [1]:
import os, sys
dr = os.path.abspath('cf_search/')
sys.path.append(dr)

from query.api import execute_query, get_default_adapter


adapter = get_default_adapter()

In [ ]:
query = """
{
    Contest {
        id 
            @filter(op_name: ">=", value: ["$min_id"]) 
            @filter(op_name: "<=", value: ["$max_id"]) 
            @output(out_name: "contest_id")
        name @output(out_name: "contest_name")
        
        out_Contest_Submission {
            id @output(out_name: "submission_id")
            programming_language 
                @filter(op_name: "=", value: ["$lang_name"])
                @output(out_name: "language")
            verdict 
                @filter(op_name: "=", value: ["$verdict"])
                @output(out_name: "verdict")
                
            out_Submission_Source {
                source_code @output(out_name: "source_code")
            }
            
            in_Submission_Problem {
                name @output(out_name: "problem_name")
                rating 
                    @filter(op_name: ">=", value: ["$min_rating"])
                    @output(out_name: "rating")
                solved_count @output(out_name: "solved_count")
            }
        }
    }
}
"""

args = {
    "min_id": 1497,
    "max_id": 1497,
    "min_rating": 1600,
    "lang_name": "Haskell",
    "verdict": "OK",
}

res = list(execute_query(adapter, query, args))
res

In [ ]:
query = """
{
    Contest {
        id 
            @filter(op_name: ">=", value: ["$min_id"]) 
            @filter(op_name: "<=", value: ["$max_id"]) 
        
        out_Contest_Submission {
            id @output(out_name: "submission_id")
            programming_language 
                @filter(op_name: "=", value: ["$lang_name"])
                @output(out_name: "language")
            verdict 
                @filter(op_name: "=", value: ["$verdict"])
                @output(out_name: "verdict")
                
            out_Submission_Source {
                out_Source_Children {
                    out_Node_Children @recurse(depth: 3) {
                        type @filter(op_name: "in_collection", value: ["$node_types"])
                        content
                            @filter(op_name: "has_substring", value: ["$node_content_str"])
                            @output(out_name: "node_content")
                    }
                }
            }
        }
    }
}
"""

args = {
    "min_id": 1470,
    "max_id": 1500,
    "lang_name": "Haskell",
    "verdict": "OK",
    "node_types": ['qualified_module'],
    "node_content_str": 'Vector',
}

for item in execute_query(adapter, query, args):
    print(item)

In [ ]:
query = """
{
    Contest {
        id 
            @filter(op_name: ">=", value: ["$min_id"]) 
            @filter(op_name: "<=", value: ["$max_id"]) 
            @output(out_name: "contest_id")
        name @output(out_name: "contest_name")
        
        out_Contest_Submission {
            id @output(out_name: "submission_id")
            programming_language 
                @output(out_name: "language")
            verdict 
                @filter(op_name: "=", value: ["$verdict"])
                @output(out_name: "verdict")
        }
    }
}
"""

args = {
    "min_id": 1497,
    "max_id": 1497,
    "min_rating": 1600,
    "verdict": "OK",
}

res = list(execute_query(adapter, query, args))
res

langs = set(map(lambda x: x['language'], res))
print(langs)

In [ ]:
query = """
{
    Problem {
        name @output(out_name: "name")
        solved_count
            @filter(op_name: "<=", value: ["$max_solves"])
            @output(out_name: "solved_count")
    }
}
"""

args = {
    "max_solves": 50,
}

res = list(execute_query(adapter, query, args))
res

In [ ]:
from tree_sitter import Language, Parser
import os

Language.build_library(
  # Store the library in the `build` directory
  'build/languages.so',

  # Include one or more languages
  [
    'vendor/tree-sitter-cpp',
    'vendor/tree-sitter-haskell',
    'vendor/tree-sitter-python'
  ]
)

In [ ]:
HASK_LANGUAGE = Language('build/languages.so', 'haskell')
parser = Parser()
parser.set_language(HASK_LANGUAGE)

hask_src = """
{-# LANGUAGE BangPatterns #-}

import Control.Monad
import qualified Data.ByteString.Char8 as B
import Data.Char
import Data.List (foldl')
-- import Debug.Trace
import Text.Printf

import Data.Maybe (fromJust)
import System.IO
import Text.Read

getNum :: IO Int
getNum = fst . fromJust . B.readInt <$> B.getLine

getNums :: IO [Int]
getNums = map ((fst . fromJust) . B.readInt) . B.split ' ' <$> B.getLine

showNums :: [Int] -> IO ()
showNums xs = putStrLn $ unwords $ map show xs

main :: IO ()
main = do
  t <- getNum
  forM_ [1 .. t] handleCase

handleCase :: Int -> IO ()
handleCase ti = do
  [n, c] <- getNums
  putStrLn $ printf "Case #%d: %s" ti (solve n c)

solve :: Int -> Int -> String
solve n c
  | c < (n - 1) || (c >= n * (n + 1) `div` 2) = "IMPOSSIBLE"
  | otherwise = construct [1..n] n c (n - 2)

construct :: [Int] -> Int -> Int -> Int -> String
construct as _ c (-1)
  | c > 0 = "IMPOSSIBLE"
  | otherwise = unwords $ map show as

construct as n c i  = construct as' n (c - stepCost) (i - 1)
  where
    stepCost = min (n - i) (c - i)
    j = i + stepCost - 1
    (p1, p1') = splitAt i as
    (p2, p3) = splitAt stepCost p1'
    as' = p1 ++ reverse p2 ++ p3

"""
hask_tree = parser.parse(bytes(hask_src, "utf8"))

In [ ]:
parser = Parser()
parser.set_language(Language('build/languages.so', 'cpp'))

cpp_src = """
#include <algorithm>
#include <iostream>
#include <numeric>
#include <vector>

using namespace std;

#define LL long long

int t, x, y;
string s;

void solve() {
  int l, r, u, d;
  l = r = u = d = 0;
  for (int i = 0; i < s.length(); i++) {
    switch (s[i]) {
    case 'R':
      r++; break;
    case 'L':
      l++;
      break;
    case 'U':
      u++;
      break;
    case 'D':
      d++;
      break;
    }
  }

  if (x > 0 && r < x) {
    cout << "NO" << endl;
    return;
  }
  if (x < 0 && l < (-x)) {
    cout << "NO" << endl;
    return;
  }

  if (y > 0 && u < y) {
    cout << "NO" << endl;
    return;
  }
  if (y < 0 && d < (-y)) {
    cout << "NO" << endl;
    return;
  }

    cout << "YES" << endl;
}

int main() {
  cin >> t;

  for (int c = 0; c < t; c++) {
    cin >> x >> y;
    cin >> s;

    solve();
  }

  return 0;
}

"""

cpp_tree = parser.parse(bytes(cpp_src, "utf8"))

In [ ]:
hask_tree.root_node.children[2].children
#print(hask_tree.root_node.children[1].children[0].walk().current_field_name())
#dir(hask_tree.root_node.children[3].children[1])

In [ ]:
cpp_tree.root_node.children[0].children

In [ ]:
node = hask_tree.root_node.children[4]
s = node.start_byte
e = node.end_byte
hask_src[s:e]

In [ ]:
print(dir(tree.root_node.sexp))
tree.root_node.sexp().format()